In [ ]:
import xarray as xr

# Load the NetCDF file
file_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc"
ds = xr.open_dataset(file_path, decode_times=False)

# Inspect the dataset
print(ds)

<xarray.Dataset> Size: 5GB
Dimensions:     (time: 86, lat: 720, lon: 1440, bounds: 2)
Coordinates:
  * time        (time) float64 688B 0.0 1.0 2.0 3.0 4.0 ... 82.0 83.0 84.0 85.0
  * lat         (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Dimensions without coordinates: bounds
Data variables: (12/17)
    primf       (time, lat, lon) float32 357MB ...
    primn       (time, lat, lon) float32 357MB ...
    secdf       (time, lat, lon) float32 357MB ...
    secdn       (time, lat, lon) float32 357MB ...
    urban       (time, lat, lon) float32 357MB ...
    c3ann       (time, lat, lon) float32 357MB ...
    ...          ...
    range       (time, lat, lon) float32 357MB ...
    secmb       (time, lat, lon) float32 357MB ...
    secma       (time, lat, lon) float32 357MB ...
    lat_bounds  (lat, bounds) float64 12kB ...
    lon_bounds  (lon, bounds) float64 23kB ...
    time_bnds   (time, bounds

In [5]:
import geopandas as gpd
import rioxarray

# Load Kenya shapefile
kenya_shp_path = "/Users/mbraaksma/Files/base_data/demetra-seals/borders/un-ocha/ken_adm_iebc_20191031_shp/ken_admbnda_adm0_iebc_20191031.shp"
kenya_boundary = gpd.read_file(kenya_shp_path)

# Ensure Kenya's CRS matches the dataset
kenya_boundary = kenya_boundary.to_crs("EPSG:4326")

# Enable rioxarray and set CRS for the dataset
ds = ds.rio.write_crs("EPSG:4326", inplace=True)

# Check for dimension names
print(ds.dims)

# Set the spatial dimensions (assuming they are 'lon' and 'lat')
ds = ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)

# Clip the dataset to the Kenya boundary
clipped_ds = ds.rio.clip(kenya_boundary.geometry, kenya_boundary.crs)

# Inspect clipped data
print(clipped_ds)


FrozenMappingWarningOnValuesAccess({'time': 86, 'lat': 720, 'lon': 1440, 'bounds': 2})


MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this. Data variable: lat_bounds

In [16]:
import xarray as xr
import geopandas as gpd
import rioxarray

# Reload the dataset
ds = xr.open_dataset(
    "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc",
    decode_times=False, decode_coords="all"
) # decode_times=False
# decode_coords="all"

ds.rio.write_crs(
    4326, inplace=True
).rio.set_spatial_dims(
    x_dim="lon", y_dim="lat", inplace=True
).rio.write_coordinate_system(inplace=True)

# Load Kenya shapefile
kenya_shp_path = "/Users/mbraaksma/Files/base_data/demetra-seals/borders/un-ocha/ken_adm_iebc_20191031_shp/ken_admbnda_adm0_iebc_20191031.shp"
kenya_boundary = gpd.read_file(kenya_shp_path)
kenya_boundary = kenya_boundary.to_crs("EPSG:4326")

# Use clip_box for a rough bounding box clip first
bbox = kenya_boundary.total_bounds  # [minx, miny, maxx, maxy]
ds_clipped_box = ds.rio.clip_box(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3])

# Then use precise clip with geometries
try:
    clipped_ds = ds_clipped_box.rio.clip(kenya_boundary.geometry, kenya_boundary.crs)
    print("Clipping successful!")
except Exception as e:
    print(f"Clip error: {e}")

# Inspect the clipped dataset
print(clipped_ds)


/var/folders/ph/3yncn97j65s8dssq7h8rh48c0000gn/T/ipykernel_14709/529721882.py:6: UserWarning: Variable(s) referenced in bounds not in variables: ['bounds_time']
  ds = xr.open_dataset(
/var/folders/ph/3yncn97j65s8dssq7h8rh48c0000gn/T/ipykernel_14709/529721882.py:6: UserWarning: Variable(s) referenced in bounds not in variables: ['bounds_lat']
  ds = xr.open_dataset(
/var/folders/ph/3yncn97j65s8dssq7h8rh48c0000gn/T/ipykernel_14709/529721882.py:6: UserWarning: Variable(s) referenced in bounds not in variables: ['bounds_lon']
  ds = xr.open_dataset(


MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this. Data variable: lat_bounds

In [18]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:      (time: 86, lat: 720, lon: 1440, bounds: 2)
Coordinates:
  * time         (time) float64 688B 0.0 1.0 2.0 3.0 4.0 ... 82.0 83.0 84.0 85.0
  * lat          (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon          (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
    spatial_ref  int64 8B 0
Dimensions without coordinates: bounds
Data variables: (12/17)
    primf        (time, lat, lon) float32 357MB ...
    primn        (time, lat, lon) float32 357MB ...
    secdf        (time, lat, lon) float32 357MB ...
    secdn        (time, lat, lon) float32 357MB ...
    urban        (time, lat, lon) float32 357MB ...
    c3ann        (time, lat, lon) float32 357MB ...
    ...           ...
    range        (time, lat, lon) float32 357MB ...
    secmb        (time, lat, lon) float32 357MB ...
    secma        (time, lat, lon) float32 357MB ...
    lat_bounds   (lat, bounds) float64 12kB ...
    lon_bounds   (lon, bounds) float64 23kB ...
    time_bnds    (time, bounds) int32 688B ...
Attributes: (12/23)
    host:                    UMD College Park
    comment:                 LUH2
    contact:                 gchurtt@umd.edu, lchini@umd.edu, steve.frolking@...
    creation_date:           2017-11-13T16:10:52Z
    title:                   UofMD LUH2f dataset prepared for input4MIPs
    activity_id:             input4MIPs
    ...                      ...
    institution:             University of Maryland (UofMD), College Park, MD...
    realm:                   land
    references:              Hurtt, Chini et al. 2011
    license:                 Land-Use Harmonization data produced by the Univ...
    target_mip:              [‘ScenarioMIP’, ‘AerChemMIP’, ‘C4MIP’, ‘DCPP’, ‘...
    source_id:               UofMD-landState-MESSAGE-ssp245-2-1-f

In [22]:
import xarray as xr
import rioxarray
import cftime

# Open the dataset with necessary parameters for handling the time variable
ds = xr.open_dataset(
    "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc",
    decode_coords="all",
    decode_times=False,  # Disable time decoding
    engine="netcdf4"  # Use netCDF4 engine for better support of non-standard calendars
)

# Convert the 'time' variable using cftime
time = cftime.num2date(ds['time'].values, units='common_years since 2015-01-01 0:0:0', calendar='noleap')

# Update the dataset with the correct time values
ds['time'] = ('time', time)

# Rename lat and lon to match expected spatial dimension names
ds = ds.rename({'lat': 'y', 'lon': 'x'})

# Set CRS and spatial dimensions
ds = ds.rio.write_crs(4326, inplace=True)
ds = ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)

# Clip the dataset using Kenya boundary
clipped_ds = ds.rio.clip(
    kenya_boundary.geometry,
    kenya_boundary.crs,
    drop=True,
    invert=False
)

# Inspect the clipped dataset
print(clipped_ds)


/var/folders/ph/3yncn97j65s8dssq7h8rh48c0000gn/T/ipykernel_14709/3772014930.py:6: UserWarning: Variable(s) referenced in bounds not in variables: ['bounds_time']
  ds = xr.open_dataset(
/var/folders/ph/3yncn97j65s8dssq7h8rh48c0000gn/T/ipykernel_14709/3772014930.py:6: UserWarning: Variable(s) referenced in bounds not in variables: ['bounds_lat']
  ds = xr.open_dataset(
/var/folders/ph/3yncn97j65s8dssq7h8rh48c0000gn/T/ipykernel_14709/3772014930.py:6: UserWarning: Variable(s) referenced in bounds not in variables: ['bounds_lon']
  ds = xr.open_dataset(


MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this. Data variable: lat_bounds

In [21]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:      (time: 86, lat: 720, lon: 1440, bounds: 2)
Coordinates:
  * time         (time) object 688B 2015-01-01 00:00:00 ... 2100-01-01 00:00:00
  * lat          (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon          (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
    spatial_ref  int64 8B 0
Dimensions without coordinates: bounds
Data variables: (12/17)
    primf        (time, lat, lon) float32 357MB ...
    primn        (time, lat, lon) float32 357MB ...
    secdf        (time, lat, lon) float32 357MB ...
    secdn        (time, lat, lon) float32 357MB ...
    urban        (time, lat, lon) float32 357MB ...
    c3ann        (time, lat, lon) float32 357MB ...
    ...           ...
    range        (time, lat, lon) float32 357MB ...
    secmb        (time, lat, lon) float32 357MB ...
    secma        (time, lat, lon) float32 357MB ...
    lat_bounds   (lat, bounds) float64 12kB ...
    lon_bounds   (lon, bounds) float64 23kB ...
    time_bnds    (time, bounds) int32 688B ...
Attributes: (12/23)
    host:                    UMD College Park
    comment:                 LUH2
    contact:                 gchurtt@umd.edu, lchini@umd.edu, steve.frolking@...
    creation_date:           2017-11-13T16:10:52Z
    title:                   UofMD LUH2f dataset prepared for input4MIPs
    activity_id:             input4MIPs
    ...                      ...
    institution:             University of Maryland (UofMD), College Park, MD...
    realm:                   land
    references:              Hurtt, Chini et al. 2011
    license:                 Land-Use Harmonization data produced by the Univ...
    target_mip:              [‘ScenarioMIP’, ‘AerChemMIP’, ‘C4MIP’, ‘DCPP’, ‘...
    source_id:               UofMD-landState-MESSAGE-ssp245-2-1-f

In [27]:
import xarray as xr

# Load the dataset with proper decoding
file_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc"
ds = xr.open_dataset(file_path, decode_times=False)

In [13]:
import xarray as xr
import cftime
from datetime import timedelta

# Load the dataset with time decoding disabled
file_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc"
ds = xr.open_dataset(file_path, decode_times=False)

# Check the time units
time_units = ds['time'].attrs['units']
print(f"Time units: {time_units}")

# Extract the start date from the time units (e.g., 'years since 2015-01-01 0:0:0')
base_year = int(time_units.split()[2].split('-')[0])  # Extract the year
start_date = cftime.DatetimeNoLeap(base_year, 1, 1)  # Use NoLeap calendar for this dataset

# Convert time values (assuming they are in 'years since...')
time_values = ds['time'].values
corrected_time = [start_date + timedelta(days=int(year * 365)) for year in time_values]

# Update the 'time' variable in the dataset
ds['time'] = xr.DataArray(corrected_time, dims='time')
print("Time updated successfully.")

# Inspect the new time variable
print(ds['time'])


Time units: years since 2015-01-01 0:0:0
Time updated successfully.
<xarray.DataArray 'time' (time: 86)> Size: 688B
array([cftime.DatetimeNoLeap(2015, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2016, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2017, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2018, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2019, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2020, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2021, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2022, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2023, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2024, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2025, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2026, 1, 1, 0, 0, 0, 0, has_year_zero=Tru

In [14]:
import geopandas as gpd
import rioxarray as rxr
from rasterstats import zonal_stats
import pandas as pd

# Load the shapefile containing the bounding box and polygons
shapefile_path = '/Users/mbraaksma/Files/base_data/demetra-seals/borders/kenya_aez.gpkg'
gdf = gpd.read_file(shapefile_path)

# Manually set CRS to WGS84 if missing
ds = ds.rio.write_crs("EPSG:4326", inplace=True)

# Rename lat/lon to y/x
ds = ds.rename({'lat': 'y', 'lon': 'x'})

# Ensure CRS match between shapefile and dataset
gdf = gdf.to_crs(ds.rio.crs)

# Clip the dataset to the bounding box (if just bbox)
bbox = gdf.total_bounds  # minx, miny, maxx, maxy
clipped_ds = ds.rio.clip_box(*bbox)

# If clipping by polygons, use:
# clipped_ds = ds.rio.clip(gdf.geometry, gdf.crs)

# Initialize an empty list to hold results
results = []

# Loop through each variable in the dataset (optional if just one)
for var in clipped_ds.data_vars:
    for i, time_val in enumerate(clipped_ds['time'].values):
        # Extract the data slice by time
        data_slice = clipped_ds[var].isel(time=i)

        # Calculate zonal stats
        stats = zonal_stats(
            gdf,
            data_slice.values,
            affine=data_slice.rio.transform(),
            categorical=True
        )

        # Append stats with time and polygon info
        for j, stat in enumerate(stats):
            stat['time'] = time_val
            stat['polygon_id'] = gdf.loc[j, 'id']  # Adjust 'id' to actual column
            results.append(stat)

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Export to CSV
csv_path = "output_zonal_stats.csv"
# df.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")


MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this. Data variable: lat_bounds

GDAL ATTEMPT

In [16]:
from osgeo import gdal
import numpy as np

# Load the NetCDF file
netcdf_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc"
dataset = gdal.Open(netcdf_path)

# Check the variables in the dataset
print(dataset.GetSubDatasets())


[('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc":primf', '[86x720x1440] area_fraction (32-bit floating-point)'), ('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc":primn', '[86x720x1440] area_fraction (32-bit floating-point)'), ('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc":secdf', '[86x720x1440] area_fraction (32-bit floating-point)'), ('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCa

In [17]:
from osgeo import ogr, osr

# Load the shapefile (polygons for zonal stats)
shapefile_path = '/Users/mbraaksma/Files/base_data/demetra-seals/borders/kenya_aez.gpkg'
shapefile = ogr.Open(shapefile_path)

# Ensure the CRS of both the shapefile and the dataset match (WGS84 is EPSG:4326)
shapefile_layer = shapefile.GetLayer()
spatial_ref = shapefile_layer.GetSpatialRef()

# Reproject dataset to the shapefile CRS if necessary (assumes shapefile is in EPSG:4326)
# You may need to check the CRS of the NetCDF dataset and reproject it here if not the same


In [18]:
# Extract data and time dimension
var_name = 'variable_name'  # Specify the variable you want to use from the NetCDF dataset
data = dataset.GetRasterBand(1)  # You may need to check which band corresponds to the variable of interest
time = dataset.GetMetadata()  # Depending on your file, the time metadata might be embedded here


In [23]:
import rasterstats as rs

# Define the polygons to calculate zonal stats
shapefile = "/path/to/shapefile.shp"  # Zonal polygons

# For each time slice in the NetCDF, calculate zonal stats
results = []
# for time_index in range(len(time)):  # Loop over time
#     data_slice = data.ReadAsArray(time_index)  # Get the data for this time slice
    
#     # Calculate zonal statistics
#     stats = rs.zonal_stats(shapefile, data_slice, categorical=True)
    
#     # Collect results (add time information)
#     for i, stat in enumerate(stats):
#         stat['time'] = time[time_index]  # Add the current time to each result
#         results.append(stat)
# Create an empty list to hold the results


# Loop over each time step
var_name = 'urban'
for time_index in range(len(ds['time'])):
    # Extract data for the current time step
    data_slice = ds[var_name].isel(time=time_index).values

    # Calculate zonal statistics
    stats = rs.zonal_stats(shapefile, data_slice, categorical=True)

    # Add time information to each result
    for i, stat in enumerate(stats):
        stat['time'] = str(ds['time'].values[time_index])  # Add time to each result
        results.append(stat)

# Convert to a DataFrame for easier processing and export
import pandas as pd
df = pd.DataFrame(results)

# Export to CSV
df.to_csv('zonal_stats_output.csv', index=False)


ValueError: Specify affine transform for numpy arrays

In [29]:
from osgeo import gdal
import numpy as np

# Open the NetCDF file using GDAL
netcdf_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc"
dataset = gdal.Open(netcdf_path)

# List all subdatasets in the NetCDF file (each dataset corresponds to a variable)
subdatasets = dataset.GetSubDatasets()
print(f"Subdatasets (variables) in the NetCDF file:")
for subdataset in subdatasets:
    print(subdataset)

# Get the first variable (usually it's the main variable of interest)
variable_ds = gdal.Open(subdatasets[0][0])  # Use the first subdataset
variable_band = variable_ds.GetRasterBand(1)

# Print basic info about the first variable
print("\nVariable Info:")
print(f"Band type: {variable_band.DataType}")
print(f"Size (rows, cols): {variable_band.XSize} x {variable_band.YSize}")

# Now, check if there's a time dimension
time_variable = dataset.GetMetadata('NC_GLOBAL')
if 'time' in time_variable:
    print(f"\nTime range: {time_variable['time']}")

# Get the time-related subdataset if available (assuming time is an independent variable)
time_ds = gdal.Open(dataset.GetSubDatasets()[0][0])  # Assuming time is the first subdataset
time_values = time_ds.ReadAsArray()

print("\nTime values from the NetCDF:")
print(time_values)


Subdatasets (variables) in the NetCDF file:
('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc":primf', '[86x720x1440] area_fraction (32-bit floating-point)')
('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc":primn', '[86x720x1440] area_fraction (32-bit floating-point)')
('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc":secdf', '[86x720x1440] area_fraction (32-bit floating-point)')
('NETCDF:"/Users/mbraaksma/Library/CloudStorage/GoogleDr

In [ ]:
from osgeo import gdal
import os

# Path to your NetCDF file
netcdf_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Files/base_data/luh2/raw_data/rcp45_ssp2/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MESSAGE-ssp245-2-1-f_gn_2015-2100.nc"
output_dir = "/Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2"

# Open the NetCDF file
dataset = gdal.Open(netcdf_path)
subdatasets = dataset.GetSubDatasets()

# Locate the 'Urban' subdataset
urban_ds_path = next(
    (subdataset[0] for subdataset in subdatasets if "urban" in subdataset[0].lower()), None
)

if not urban_ds_path:
    print("Urban variable not found.")
else:
    # Open the Urban subdataset
    urban_ds = gdal.Open(urban_ds_path)
    num_bands = urban_ds.RasterCount

    for band_index in range(1, num_bands + 1):
        urban_band = urban_ds.GetRasterBand(band_index)

        # Handle nodata/fill value
        nodata_value = urban_band.GetNoDataValue() or 1e+20  # Default if not set

        # Extract data array and replace nodata values with NaN
        data_array = urban_band.ReadAsArray()
        data_array[data_array >= nodata_value] = float('nan')

        # Extract year
        time_metadata = urban_band.GetMetadata().get("NETCDF_DIM_time", "unknown")
        years_since_2015 = int(time_metadata) if time_metadata.isdigit() else f"band{band_index}"
        year = years_since_2015 + 2015
        
        if year < 2031:        
            # Output file path
            output_file = os.path.join(output_dir, f"rcp45_ssp2_urban_{year}.tif")

            # Create output dataset for a single band
            driver = gdal.GetDriverByName("GTiff")
            output_ds = driver.Create(
                output_file,
                urban_ds.RasterXSize,
                urban_ds.RasterYSize,
                1,  # Single band
                gdal.GDT_Float32,  # Ensure float output
                ["COMPRESS=LZW"]
            )

            # Copy geotransform and projection from original dataset
            output_ds.SetGeoTransform(urban_ds.GetGeoTransform())
            output_ds.SetProjection(urban_ds.GetProjection())

            # Write the corrected data array
            output_ds.GetRasterBand(1).WriteArray(data_array)
            output_ds.GetRasterBand(1).SetNoDataValue(float('nan'))

            output_ds.FlushCache()
            output_ds = None
            print(f"Exported: {output_file}")

    urban_ds = None


Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2015.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2016.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2017.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2018.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2019.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2020.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2021.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2022.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2023.tif
Exported: /Users/mbraaksma/Files/base_data/demetra-seals/lulc/lu

In [39]:
# Locate and open the 'Urban' subdataset (adjust as necessary)
urban_ds_path = next((sub[0] for sub in subdatasets if "urban" in sub[0].lower()), None)
urban_ds = gdal.Open(urban_ds_path)
netcdf_band = urban_ds.GetRasterBand(1)  # 2015 assumed as band 1
netcdf_data = netcdf_band.ReadAsArray()

# Open the GeoTIFF file
geotiff_ds = gdal.Open('/Users/mbraaksma/Files/base_data/demetra-seals/lulc/luh2/rcp45_ssp2/rcp45_ssp2_urban_2015.tif')
geotiff_band = geotiff_ds.GetRasterBand(1)
geotiff_data = geotiff_band.ReadAsArray()

# Check for statistics and differences
netcdf_data_masked = np.ma.masked_invalid(netcdf_data)
geotiff_data_masked = np.ma.masked_invalid(geotiff_data)

netcdf_stats = {
    "mean": netcdf_data_masked.mean(),
    "min": netcdf_data_masked.min(),
    "max": netcdf_data_masked.max(),
}

geotiff_stats = {
    "mean": geotiff_data_masked.mean(),
    "min": geotiff_data_masked.min(),
    "max": geotiff_data_masked.max(),
}

difference = np.ma.sum(np.abs(netcdf_data_masked - geotiff_data_masked))

print("NetCDF Stats:", netcdf_stats)
print("GeoTIFF Stats:", geotiff_stats)
print("Total Difference:", difference)

NetCDF Stats: {'mean': 7.506379238119398e+19, 'min': 0.0, 'max': 1e+20}
GeoTIFF Stats: {'mean': 0.0036410985065962, 'min': 0.0, 'max': 1.0}
Total Difference: 0.0
